In [13]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import os

def file_txt_to_sql(og_filename, filename, table):
    chunk_size = 10_000
    og_file = f'/Users/mmunozcampos/Documents/Comp y reg/{og_filename}'
    db_path = f"/Users/mmunozcampos/Documents/Comp y reg/{filename}"
    
    # Nombres de las columnas
    column_names = [
        'codigo_aseguradora','periodo_informacion', 'tipo_registro', 'run_beneficiario',
        'sexo_beneficiario', 'edad_beneficiario', 'tipo_beneficiario', 'rut_prestador',
        'region_prestador', 'prestador_preferente', 'programa_medico_principal',
        'programa_medico_complementario', 'bono_atencion', 'reembolso',
        'codigo_prestacion', 'pertenencia_codigo',
        'cobertura_financiamiento_prestacion', 'fecha_bonificacion',
        'tipo_prestador', 'tipo_atencion', 'frecuencia', 'valor_facturado',
        'valor_bonificado', 'monto_copago', 'bonificacion_restringida',
        'tipo_plan', 'modalidad_intervencion_quirurgica',
        'ley_urgencia_vital', 'codigo_gpp', 'identificacion_ges', 'none'
    ]
    
    # Creamos SQLite engine con extended timeout y pragma settings
    engine = create_engine(f"sqlite:///{db_path}", 
                          connect_args={"timeout": 300, "check_same_thread": False})
    
    # Prevención de problemas de tipeo
    conn = sqlite3.connect(db_path)
    conn.execute("PRAGMA journal_mode = WAL;")  # Write-Ahead Logging for better performance
    conn.execute("PRAGMA synchronous = NORMAL;")  # Less durability but faster performance
    conn.close()
    
    cols_numericas = [
        'codigo_aseguradora','run_beneficiario', 'rut_prestador',
        'region_prestador', 'programa_medico_principal',
        'programa_medico_complementario', 'bono_atencion', 'reembolso',
        'frecuencia', 'valor_facturado',
        'valor_bonificado', 'monto_copago'
    ]

    #preparamos el porcentaje de completado
    total_lines = sum(1 for _ in open(og_file, encoding='latin1'))
    total_chunks = total_lines // chunk_size + 1

    # Correr en chunks 
    for i, chunk in enumerate(pd.read_csv(og_file, delimiter='|', 
                                          names=column_names, 
                                          header=None, 
                                          chunksize=chunk_size, 
                                          encoding='latin1' 
                                         )):
        #Nos encargamos de las columnas numericas
        for col in cols_numericas:
            if 'object' in str(chunk[col].dtype):
                chunk[col] = chunk[col].str.strip()
                chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
        #exportamos a sql
        chunk.iloc[:, :-1].to_sql(table, con=engine, if_exists='append', index=False)

        percent_complete = (i + 1) / total_chunks * 100
        print(f"Cargando: {percent_complete:.2f}% completo")
        
        #para testeo
        #if i == 0:
        #    break
        
    engine.dispose()
    return print(f'{filename} fue un exito')

In [7]:
file_txt_to_sql("Prestaciones_Bonificadas_2024t1.txt", "prestaciones_t2024.db", "tabla2024")

prestaciones_t2024.db fue un exito


In [8]:
file_txt_to_sql("Prestaciones_Bonificadas_2024t2.txt", "prestaciones_t2024.db", "tabla2024")

prestaciones_t2024.db fue un exito


In [11]:
file_txt_to_sql("Prestaciones_Bonificadas_2024t3.txt", "prestaciones_t2024.db", "tabla2024")

Cargando: 0.06% completo
Cargando: 0.11% completo
Cargando: 0.17% completo
Cargando: 0.23% completo
Cargando: 0.28% completo
Cargando: 0.34% completo
Cargando: 0.40% completo
Cargando: 0.45% completo
Cargando: 0.51% completo
Cargando: 0.57% completo
Cargando: 0.62% completo
Cargando: 0.68% completo
Cargando: 0.74% completo
Cargando: 0.79% completo
Cargando: 0.85% completo
Cargando: 0.91% completo
Cargando: 0.96% completo
Cargando: 1.02% completo
Cargando: 1.08% completo
Cargando: 1.13% completo
Cargando: 1.19% completo
Cargando: 1.25% completo
Cargando: 1.30% completo
Cargando: 1.36% completo
Cargando: 1.41% completo
Cargando: 1.47% completo
Cargando: 1.53% completo
Cargando: 1.58% completo
Cargando: 1.64% completo
Cargando: 1.70% completo
Cargando: 1.75% completo
Cargando: 1.81% completo
Cargando: 1.87% completo
Cargando: 1.92% completo
Cargando: 1.98% completo
Cargando: 2.04% completo
Cargando: 2.09% completo
Cargando: 2.15% completo
Cargando: 2.21% completo
Cargando: 2.26% completo


In [14]:
file_txt_to_sql("Prestaciones_Bonificadas_2024t4.txt", "prestaciones_t2024.db", "tabla2024")

Procesando prestaciones_t2024.db:   0%|        | 1/1822 [00:00<09:26,  3.22it/s]

Cargando: 0.05% completo


Procesando prestaciones_t2024.db:   0%|        | 2/1822 [00:00<09:58,  3.04it/s]

Cargando: 0.11% completo


Procesando prestaciones_t2024.db:   0%|        | 3/1822 [00:00<09:36,  3.16it/s]

...

Cargando: 99.84% completo


Procesando prestaciones_t2024.db: 100%|████▉| 1820/1822 [08:39<00:00,  3.49it/s]

Cargando: 99.89% completo


Procesando prestaciones_t2024.db: 100%|█████| 1822/1822 [08:39<00:00,  3.50it/s]

Cargando: 99.95% completo
Cargando: 100.00% completo
prestaciones_t2024.db fue un exito


In [2]:
import sqlite3
import pandas as pd

db_path = "/Users/mmunozcampos/Documents/Comp y reg/prestaciones_t2024.db"
conn = sqlite3.connect(db_path)
query = ''' SELECT rut_prestador, SUM(frecuencia) AS frecuencia_total
        FROM tabla2024 
        WHERE tipo_atencion == "Atención Hospitalaria" AND region_prestador == 13
        GROUP BY rut_prestador
        ORDER BY frecuencia_total DESC
        LIMIT 8; '''
		
top8_rm = pd.read_sql(query, conn)
top8_rm

,rut_prestador,frecuencia_total
0,628513,2263154
1,1324822,1745725
2,3325777,1558533
3,1159334,1331688
4,2868186,1117671
5,422649,813265
6,27564,697456
7,922214,561704


In [3]:
query = ''' SELECT 
    rut_prestador,
    SUM(frecuencia) * 100.0 / (
        SELECT SUM(frecuencia)
        FROM tabla2024
        WHERE region_prestador = 13
          AND tipo_atencion = "Atención Hospitalaria"
    ) AS market_share_percentage
FROM tabla2024
WHERE region_prestador = 13
  AND tipo_atencion = "Atención Hospitalaria"
GROUP BY rut_prestador
ORDER BY market_share_percentage DESC
LIMIT 8;

'''
top8_marketshare = pd.read_sql(query, conn)
top8_marketshare

,rut_prestador,market_share_percentage
0,628513,15.901149
1,1324822,12.265641
2,3325777,10.950411
3,1159334,9.356575
4,2868186,7.852870
5,422649,5.714082
6,27564,4.900397
7,922214,3.946589


In [37]:
query = ''' SELECT codigo_prestacion, SUM(frecuencia) AS frecuencia_total
        FROM tabla2024
        WHERE tipo_atencion == "Atención Hospitalaria"
        AND region_prestador = 13
        AND substr(codigo_prestacion, 1, 2) = '11'
        AND length(codigo_prestacion) > 3
        GROUP BY codigo_prestacion
        ORDER BY frecuencia_total DESC
        LIMIT 5; '''
top5_neuro = pd.read_sql(query, conn)
top5_neuro

,codigo_prestacion,frecuencia_total
0,1101011,6155
1,1101050,5500
2,1103048,4209
3,1101043,3586
4,1101045,3514


In [35]:
query = ''' SELECT codigo_prestacion
        FROM (SELECT * FROM tabla2024 LIMIT 100000)
        WHERE tipo_atencion == "Atención Hospitalaria"
        AND region_prestador = 13
        AND length(codigo_prestacion) <= 3
        GROUP BY codigo_prestacion
        LIMIT 300; '''
pd.read_sql(query, conn)

,codigo_prestacion
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [36]:
query = ''' SELECT codigo_prestacion
        FROM tabla2024
        WHERE tipo_atencion == "Atención Hospitalaria"
        AND region_prestador = 13
        AND length(codigo_prestacion) <= 3
        GROUP BY codigo_prestacion
        LIMIT 300; '''
pd.read_sql(query, conn)

,codigo_prestacion
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [43]:
conn.close()

In [10]:
df_fonasa = pd.read_excel("Códigos Fonasa.xlsx")

In [29]:
df_fonasa[df_fonasa["grupo"] == 11]

,código,código (4),código (c/guarismo) para merge,grupo,sub_grupo,presta,equipo,pab,Num,glosa,Especialidad,Sub especialidad
859,1101001,1101,110100100,11,1,1,0,0,0.0,RELLENADO DE BOMBAS DE ADMINISTRACION DE FARM...,Neurología y neurocirugía,NaN
860,1101002,1101,110100200,11,1,2,0,0,0.0,PUNCION SUBDURAL ...,Neurología y neurocirugía,NaN
861,1101003,1101,110100300,11,1,3,0,0,0.0,PUNCION LUMBAR C/S MANOMETRIA C/S QUECKENSTED ...,Neurología y neurocirugía,NaN
862,1101140,1101,110114000,11,1,140,0,0,0.0,"ESCLEROSIS MULTIPLE REMITENTE RECURRENTE, TRA...",Neurología y neurocirugía,NaN
863,1101141,1101,110114100,11,1,141,0,0,0.0,"ESCLEROSIS MULTIPLE REMITENTE RECURRENTE, TRAT...",Neurología y neurocirugía,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
961,1103064,1103,110306408,11,3,64,2,8,8.0,"SINDROME DEL ESCALENO, TRAT. QUIR. ...",Neurología y neurocirugía,NaN
962,1103065,1103,110306508,11,3,65,2,8,8.0,"SINDROME DE COSTILLA CERVICAL, TRAT. QUIR. ...",Neurología y neurocirugía,NaN
963,1103066,1103,110306607,11,3,66,2,7,7.0,LIBERACION QUIRURGICA DE NERVIO PERIFERICO EXT...,Neurología y neurocirugía,NaN
964,1103067,1103,110306707,11,3,67,2,7,7.0,"LIBERACION DE NERVIO CUBITAL A NIVEL DEL CODO,...",Neurología y neurocirugía,NaN


In [48]:
for cod in top5_neuro["codigo_prestacion"].unique():
    print(cod)
    print(df_fonasa.loc[df_fonasa['código'] == cod]["glosa"])

1101011
879    POTENCIALES EVOCADOS EN CORTEZA ( POR EJ.: AUD...
Name: glosa, dtype: object
1101050
Series([], Name: glosa, dtype: object)
1103048
943    INFILTRACION FACETARIA COLUMNA Y RADICULAR    ...
Name: glosa, dtype: object
1101043
873    EEG DIGITAL 32 CANALES (CON ACTIVACIONES HV Y ...
Name: glosa, dtype: object
1101045
876    POLISOMNOGRAFIA  (ELECTROENCEFALOGRAMA, ELECTR...
Name: glosa, dtype: object


In [42]:
df_fonasa['grupo'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29])

In [49]:
df_fonasa[df_fonasa["código"] == 1101050]

,código,código (4),código (c/guarismo) para merge,grupo,sub_grupo,presta,equipo,pab,Num,glosa,Especialidad,Sub especialidad


In [51]:
df_fonasa[df_fonasa["código (c/guarismo) para merge"] == 101745]

,código,código (4),código (c/guarismo) para merge,grupo,sub_grupo,presta,equipo,pab,Num,glosa,Especialidad,Sub especialidad
